In [143]:
import sqlite3
import csv
import pandas as pd
import os
import re

import math
#from nltk import word_tokenize, sent_tokenize # widely used text tokenizer
from nltk.stem.porter import PorterStemmer # an approximate method of stemming words
stemmer = PorterStemmer()

from difflib import SequenceMatcher as SeqMatcher
import numpy as np

folder_prefix = '/home/jovyan/work/'

In [144]:
new_data = pd.read_pickle(folder_prefix+ 'charters_full_2015.pkl')

In [145]:
keywords = ['values', 'academics', 'skills', 'purpose',
                       'direction', 'mission', 'vision', 'vision', 'mission', 'our purpose',
                       'our ideals', 'ideals', 'our cause', 'curriculum','curricular',
                       'method', 'pedagogy', 'pedagogical', 'approach', 'model', 'system',
                       'structure','philosophy', 'philosophical', 'beliefs', 'believe',
                       'principles', 'creed', 'credo', 'values','moral', 'history', 'our story',
                       'the story', 'school story', 'background', 'founding', 'founded',
                       'established','establishment', 'our school began', 'we began',
                       'doors opened', 'school opened', 'about us', 'our school', 'who we are',
                       'our identity', 'profile', 'highlights']

mission_keywords = ['mission','vision', 'vision:', 'mission:', 'our purpose', 'our ideals', 'ideals:', 'our cause', 'cause:', 'goals', 'objective']
curriculum_keywords = ['curriculum', 'curricular', 'program', 'method', 'pedagogy', 'pedagogical', 'approach', 'model', 'system', 'structure']
philosophy_keywords = ['philosophy', 'philosophical', 'beliefs', 'believe', 'principles', 'creed', 'credo', 'value',  'moral']
history_keywords = ['history', 'story','our story', 'the story', 'school story', 'background', 'founding', 'founded', 'established', 'establishment', 'our school began', 'we began', 'doors opened', 'school opened']
about_keywords =  ['about us', 'our school', 'who we are', 'overview', 'general information', 'our identity', 'profile', 'highlights']

mission_keywords = set(stemmer.stem(word) for word in mission_keywords)
curriculum_keywords = set(stemmer.stem(word) for word in curriculum_keywords)
philosophy_keywords = set(stemmer.stem(word) for word in philosophy_keywords)
history_keywords = set(stemmer.stem(word) for word in history_keywords)
about_keywords =  set(stemmer.stem(word) for word in about_keywords)


In [232]:
# new_data

In [233]:
# loc = new_data.columns.get_loc('WEBTEXT')
# new_data.iloc[:, [loc]]

In [234]:
# #all_pages is a list of strings of all web page texts
# all_pages = []

# no_nan_data = new_data.[0:7]dropna(subset=['WEBTEXT'])
# col_pages = no_nan_data['WEBTEXT'] #change to new_data['data'] later, but work with first 1 schools for now
# i = 0
# count = 0
# for school_data in col_pages:

#     for tup in school_data:
#         all_pages.append(tup[3])
      

#     i+=1

In [235]:
# new_data['WEBTEXT'][0]

In [236]:
def create_sim_df(pages):
    
    ratio_df = pd.DataFrame(np.array(pages), columns=["Page"])
    ratio_df["Ratios"] = np.nan
    ratio_df["Similar"] = np.nan #similar will hold similar indexes, > 0.90 ratios

    ratio_df['Ratios'] = ratio_df['Ratios'].astype('object')
    ratio_df['Similar'] = ratio_df['Similar'].astype('object')
    
    index = 0
    for page0 in pages:
        ratios_list = []
        for page1 in pages:
            ratios_list.append(SeqMatcher(None, page0, page1).ratio())
        ratio_df['Ratios'][index] = ratios_list
        sim_ind_list = np.asarray(np.where((np.asarray(ratios_list) >= 0.5) & (np.asarray(ratios_list) != 1.0))[0]).tolist()
        ratio_df['Similar'][index] = sim_ind_list
        index+=1
                
    num_rows = ratio_df.shape[0] 
    tot_tuples = []

    final_cut_strings = [None] * num_rows
    count_not_similar = 0  
    not_sim_list = []
    sim_list = []

    for r in range(num_rows):
        if (len(ratio_df['Similar'][r]) != 0):
            sim_list.append(r)
            #print("has similar > 0.9 at index : " + str(r))
            list_cut = []
            for ind in ratio_df['Similar'][r]:
                list_of_triples = SeqMatcher(None, ratio_df['Page'][r], ratio_df['Page'][ind]).get_matching_blocks()
                zeroth_triple = list_of_triples[0] #first triple, most likely in beginning, most likely a header
                n = zeroth_triple[2] #j to j+n are the indices of the overlapping part
                orig_string = ratio_df['Page'][ind]
                cut_down_string = orig_string[:zeroth_triple[1]] + orig_string[zeroth_triple[1] + n:] #removes overlapping part
                list_cut.append([ind, cut_down_string])
            tot_tuples.extend(list_cut) #list of tuples
        else:
            #print("no similar at index : " + str(r))
            not_sim_list.append(r)
            count_not_similar +=1
            final_cut_strings[r]= ratio_df['Page'][r]
    
    return tot_tuples, final_cut_strings, ratio_df


In [237]:
# index = 0
# for page0 in unique_pages_set:
#     ratios_list = []
#     for page1 in unique_pages_set:
#         ratios_list.append(SeqMatcher(None, page0, page1).ratio())
#     ratio_df['Ratios'][index] = ratios_list
#     sim_ind_list = np.asarray(np.where((np.asarray(ratios_list) > 0.9) & (np.asarray(ratios_list) != 1.0))[0]).tolist()
#     ratio_df['Similar'][index] = sim_ind_list
#     index+=1

In [238]:
# num_rows = ratio_df.shape[0] 
# tot_tuples = []

# final_cut_strings = [None] * num_rows
# count_not_similar = 0  
# not_sim_list = []
# sim_list = []

# for r in range(num_rows):
#     if (len(ratio_df['Similar'][r]) != 0):
#         sim_list.append(r)
#         #print("has similar > 0.9 at index : " + str(r))
#         list_cut = []
#         for ind in ratio_df['Similar'][r]:
#             list_of_triples = SeqMatcher(None, ratio_df['Page'][r], ratio_df['Page'][ind]).get_matching_blocks()
#             zeroth_triple = list_of_triples[0] #first triple, most likely in beginning, most likely a header
#             n = zeroth_triple[2] #j to j+n are the indices of the overlapping part
#             orig_string = ratio_df['Page'][ind]
#             cut_down_string = orig_string[:zeroth_triple[1]] + orig_string[zeroth_triple[1] + n:] #removes overlapping part
#             list_cut.append([ind, cut_down_string])
#         tot_tuples.extend(list_cut) #list of tuples
#     else:
#         #print("no similar at index : " + str(r))
#         not_sim_list.append(r)
#         count_not_similar +=1
#         final_cut_strings[r]= ratio_df['Page'][r]

In [239]:
def create_first_header_cut(tot_tuples, final_cut_strings, num_rows):
    #num_rows = ratio_df.shape[0]
    list_grouped = [[] for x in range(num_rows)] #big list, just put list of strings in spots where needed
    list_indices_of_groups = []
    for ind in range(num_rows):
        for tup in tot_tuples:
            if(tup[0] == ind):
                list_grouped[ind].append(tup[1]) #attach that tuple's string
                list_indices_of_groups.append(tup[0])
                
    ind_fill_final_grouped = []
    i = 0
    for group in list_grouped:
        if (len(group) != 0):
            ind_fill_final_grouped.append(i)
    
        i+=1
        
    spot = 0
    #add into final cut strings, the new "cut down" versions of appropriate strings
    for li in list_grouped:
        if (len(li) != 0):
            #print(unique_ind_list[spot])
            final_cut_strings[ind_fill_final_grouped[spot]]= min(li, key=len) #inserts into correct index, what was None before, add in that string now
            spot+=1
    
    return final_cut_strings

In [240]:
# list_grouped = [[] for x in range(num_rows)] #big list, just put list of strings in spots where needed
# list_indices_of_groups = []
# for ind in range(num_rows):
#     for tup in tot_tuples:
#         if(tup[0] == ind):
#             list_grouped[ind].append(tup[1]) #attach that tuple's string
#             list_indices_of_groups.append(tup[0])

In [241]:
#not used
#unique_group_ind = set(list_indices_of_groups) 

In [242]:
# ind_fill_final_grouped = []
# i = 0
# for group in list_grouped:
#     if (len(group) != 0):
#         #print("list ready to insert at index " + str(i) + "\n")
#         ind_fill_final_grouped.append(i)
    
#     i+=1

In [243]:
# spot = 0
# #add into final cut strings, the new "cut down" versions of appropriate strings
# for li in list_grouped:
#     if (len(li) != 0):
#         #print(unique_ind_list[spot])
#         final_cut_strings[ind_fill_final_grouped[spot]]= min(li, key=len) #inserts into correct index, what was None before, add in that string now
#         spot+=1

In [244]:
def create_second_header_cut(first_header_cut):
    
    super_final_strings = []
    for s in first_header_cut:
        s_new = ""
        if(s is None):
            s_new = ""
        else:
            s_new = s
        use_punc = False
        use_sev = False
        punc = [",", ".", ":", ";"]
        p_list = []
        for p in punc:
            if (s_new.find(p) != -1):
                p_list.append(s_new.find(p))
            else:
                p_list.append(len(s_new))

        punc_ind = min(p_list)

        n_list = [index for index, k in enumerate(s_new) if k=='\n']
        start_punc = len(s_new)
        for i in n_list:
            if(i < punc_ind):
                start_punc = i # start_punc equals the largest index of \n that's less than index of first punctuation

        start = 0
        end = 0
        total = ""
        list_totals = []
        st_en = []
        for c in s_new:
            if (c not in ['\n', '\t']):
                total+=(c)
                end+=1
            else :
                if(len(total.split()) >= 7): # we hit 7 words or more, wipe everything before start index
                    #list_totals.append(total)
                    st_en.append((start, end))

                total= ""
                start = end
        start_sev = len(s_new)-1 #len(s)-1 #make it huge by default; if there's no group of 7, then start punc will be the smallest
        if len(st_en) > 0:
            start_sev = st_en[0][0] #index of first group of words that's >= 7 words; 0th tuple's start value


        #take smaller of the two indices, since we want to use the property which occurs first
        if start_punc < start_sev:
            #if start of sentence which ends in/contains puncuation occurs earlier, wipe eveything before that index
            #only take that index +1 and on, start right after the new line
            new_string = s_new[start_punc+1:] 
            super_final_strings.append(new_string)

        else:
            #if start of group of words that >= 7 occurs earlier than a sentence with punctuation, wipe eveything before that index
            #only take that index and on, statr using that begining of the group of 7+ words
            new_string = s_new[start_sev:] 
            super_final_strings.append(new_string)
        
    return super_final_strings



In [245]:
#first removal of headers in final_cut_Strings currently, but now we want to cut down headers more
#take out text before the first sentence or text before the first group of 7+ words

# super_final_strings = []
# for s in final_cut_strings:
#     use_punc = False
#     use_sev = False
#     punc = [",", ".", ":", ";"]
#     p_list = []
#     for p in punc:
#         if (s.find(p) != -1):
#             p_list.append(s.find(p))
#         else:
#             p_list.append(len(s))
    
#     punc_ind = min(p_list)
    
#     n_list = [index for index, k in enumerate(s) if k=='\n']
#     start_punc = len(s)
#     for i in n_list:
#         if(i < punc_ind):
#             start_punc = i # start_punc equals the largest index of \n that's less than index of first punctuation
    
#     start = 0
#     end = 0
#     total = ""
#     list_totals = []
#     st_en = []
#     for c in s:
#         if (c not in ['\n', '\t']):
#             total+=(c)
#             end+=1
#         else :
#             if(len(total.split()) >= 7): # we hit 7 words or more, wipe everything before start index
#                 #list_totals.append(total)
#                 st_en.append((start, end))
    
#             total= ""
#             start = end
#     start_sev = len(s)-1 #len(s)-1 #make it huge by default; if there's no group of 7, then start punc will be the smallest
#     if len(st_en) > 0:
#         start_sev = st_en[0][0] #index of first group of words that's >= 7 words; 0th tuple's start value
   

#     #take smaller of the two indices, since we want to use the property which occurs first
#     if start_punc < start_sev:
#         #if start of sentence which ends in/contains puncuation occurs earlier, wipe eveything before that index
#         #only take that index +1 and on, start right after the new line
#         new_string = s[start_punc+1:] 
#         super_final_strings.append(new_string)
        
#     else:
#         #if start of group of words that >= 7 occurs earlier than a sentence with punctuation, wipe eveything before that index
#         #only take that index and on, statr using that begining of the group of 7+ words
#         new_string = s[start_sev:] 
#         super_final_strings.append(new_string)
        
     
            

In [246]:
# #list of common words in footers
# footer_list = ["Copyright", "All Rights Reserved",  "Read More", 
#                "Useful Links", "Search", "Survey", "Feed", "Phone", "Fax", "Address",  "Sitemap", 
#                "Jobs", "Apply", "Pre-Enroll"]
# #facebook, contact us, enroll etc occurs in headers as well, so had to take that out

# footers_removed_strings = []

# #look for the keyword in each string and if found, remove all the text after it
# for s in super_final_strings:
#     no_newline = s.replace("\n", " ")
#     new_list = []
#     for word in footer_list:
#         if (no_newline.find(word) != -1):
#             new_list.append(no_newline.find(word))
#         else :
#             new_list.append(len(s))

#     #get the index of the earliest occurence of a keyword
#     f_ind = min(new_list)
    
#     #go back to new line or period right before and wipe out everything after that
#     n_list = [index for index, k in enumerate(s) if k in ['\n', '.']]
#     start_punc = len(s)
#     for i in n_list:
#         if(i < f_ind):
#             start_punc = i # start_punc equals the largest index of \n or . that's less than index of the keyword
            
#     if start_punc < f_ind:
#             footer_rem = s[:start_punc]
#             footers_removed_strings.append(footer_rem) 
#     else:
#             footer_rem = s[:f_ind]
#             footers_removed_strings.append(footer_rem) 
              

In [247]:
#footers_removed_strings

In [248]:
#footers_removed_strings are the final, most cut down versions of the web text
#we did process on "WEBTEXT" column of new_data
#now repeat for "CMO_WEBTEXT" column of new_data

In [249]:
# cleaned_strings = pd.DataFrame(footers_removed_strings, columns=["Cleaned Text"])
# cleaned_strings.to_csv('CleanedStrings.csv', index=False)

In [250]:
#compare between pages of each school

def remove_string_overlaps(tuplist):
   
    unique_tuplist = []
    seen_pages = set() # Initialize list of known pages for a school
    unique_pages=[]
    reversed_pages = []
    tup_indices = []

    cleaned_strings = []
    for tup in tuplist:
        seen_pages.add(tup[3])

    for i in range(len(tuplist)):
        if(tuplist[i][3] in seen_pages and (tuplist[i][3]  not in unique_pages) and (tuplist[i][3] is not None)):
            unique_tuplist.append(tuplist[i])
            unique_pages.append(tuplist[i][3])
            reversed_pages.append(tuplist[i][3][::-1])
            tup_indices.append(i)
            #print("unique page : " + str(i))

    #now compare all pages with each other 
    #print(unique_tuplist)
    tot_tuples, final_cut_strings, ratio_df = create_sim_df(unique_pages)
    first_header_cut = create_first_header_cut(tot_tuples, final_cut_strings, ratio_df.shape[0])
    
    #first removal of headers in final_cut_Strings currently, but now we want to cut down headers more
    #take out text before the first sentence or text before the first group of 7+ words  
    second_header_cut = create_second_header_cut(first_header_cut)
    
    
    #now run process on reversed strings
    
    rev_tot_tuples, rev_final_cut_strings, rev_ratio_df = create_sim_df(reversed_pages)
    
    rev_first_header_cut = create_first_header_cut(rev_tot_tuples, rev_final_cut_strings, rev_ratio_df.shape[0])
    
    for i in range(len(rev_first_header_cut)):
        #find where to cut the footer off , index
        
        add_string = second_header_cut[i]
        if i in tup_indices: 
            forward_string = rev_first_header_cut[i][::-1]
            #print(forward_string)
            sept = int(len(forward_string)/2)
            half_string = forward_string[len(forward_string) - sept:]
            #find that half in the regular string, and get the end of the half
            #print(type(second_header_cut[i]))
            #print(type(half_string))
            end_index = (second_header_cut[i]).find(half_string) + len(half_string) - 1
            add_string = second_header_cut[i][:end_index]
            #remove the footer aka remove stuff after the end_index
            #keep the stuff before end_index
        
        cleaned_strings.append(add_string)


    #         for tup in tuplist: # Iterate over tuples in tuplist (list of tuples)
    #             if tup[3] in known_pages or tup=='': # Compare this page with previous ones
    #             continue # Or better yet, remove tuple
    #             # CLEAN STRING HERE
    #             known_pages.add(tup[3]) # Add this page to pages we've already seen
    
    #then iterate through cleaned_strings and inset into each tuple
    new_list = []
    for count in range(len(cleaned_strings)):
        new_tup = (tuplist[tup_indices[count]][0], tuplist[tup_indices[count]][1], tuplist[tup_indices[count]][2], cleaned_strings[count])
        new_list.append(new_tup)
#         print(tup_indices[count])
#         print(cleaned_strings[count])
#         print("\n  \n")

#     print(len(new_list))
#     print(len(tup_indices))
    return new_list


#first make a list of tuples that has no nan values and has
#if not np.isnan(tuplist):   CHECK FOR NAN OUTSIDE

#k = remove_string_overlaps(new_data['WEBTEXT'][11])


In [251]:
#apply remove_string_overlaps on each school, aka on each row of new_data
#since pages of a school will likely be similar to the other pages within that school own 

In [253]:
def parse_df(old_list):
    #print(loc)
    new_list = remove_string_overlaps(old_list)
    return new_list
    
# loc = new_data.columns.get_loc('WEBTEXT')
# # new_data.iloc[:, [loc]]
# temp = new_data.iloc[11:13]
# temp['WEBTEXT'] = temp['WEBTEXT'].apply(parse_df)
# temp['WEBTEXT'][0]

new_data['WEBTEXT'] = new_data['WEBTEXT'].apply(parse_df)
new_data.to_pickle('new_removed_overlaps.pkl')

In [223]:
# k = [1,2,3,4]

# def multiply(li):
#     return [3*x for x in li]

# def plus_2(old_list):
#     new_list = multiply(old_list) 
#     old_list = new_list
#    # print(old_list)
#     #print(new_list)
    
# k= plus_2(k)
# k

In [224]:
# new_data['WEBTEXT'][11]

In [225]:
#k

In [226]:
# s = "Skip to Main Content\nToggle main menu visibility \nHome\nAbout Us\nAbout Our School\nOur History\nAdministration\nBoard Members\nSchools\nHouston Elementary\nHouston Middle School\nLancaster Campus\nRegistration\nCalendar\nNews\nDepartments\nPreschool\nFood Services\nHuman Resources\nHelpful Links\nContact Us\nACCELERATED INTERDISCIPLINARY INTERMEDIATE ACADEMY\n“The school where every challenge becomes a mission accomplished.\n”\nContact Us\nWe at AIA welcome your questions and comments. Please feel free to contact us via e-mail or using the methods listed below.\nOur Address\nAIA District\nP.O. Box 20589\nHouston, Texas 77225-0589\nAIA District E-mail\nOur Houston Locations\nElementary School\n12825 Summit Ridge\nHouston, TX 77085\nPhone: (713) 728-9330\nMiddle School\n12825 Summit Ridge\nHouston, TX 77085\nPhone: (713) 283-6298\nOur Lancaster Location\nElementary School\n901 East Beltline Rd.\nLancaster, TX 75146\nPhone: (972) 227-2105\n*Note: E-mail resumes and enrollment information for AIA Lancaster Elem to the \nHuman Resources Department\n.\nOur Office Hours\n7:15 a.m. - 5:00 p.m. Monday through Friday\nParent Resources\nParent and Student Handbook\nEnrollment Forms\nRegistration\nSchool Menus\nOur Schools\n\r\t\t\t\t\t\tCopyright ©2018\xa0| Site designed and maintained by \nSchool Webmasters"
# half_string = s[int(len(s)/2):]
# end_index = s.find(half_string) + len(half_string) - 1
# end_index

In [227]:
# new_data['WEBTEXT'][9][4]

In [228]:
# remove_string_overlaps(new_data['WEBTEXT'][9])[3]

In [229]:
# tuplist = new_data['WEBTEXT'][0]
# seen_pages = set()
# k = []
# for tup in tuplist:
#     seen_pages.add(tup[3])

# index = 0
# for tup in tuplist:
#     if tup[3] in seen_pages and tup[3] not in k:
#         k.append(tup[3])
#         print(index)
#     else:
#         print("the repeated indices : " + str(index))
#     index+=1
# #len(seen_pages)

In [230]:
# tuplist[0][3]

In [231]:
# tuplist[1][3]